In [ ]:
import torch
import pandas as pd
import numpy as np
import glob
import os
import gc
from torch_geometric.data import HeteroData
from sklearn.preprocessing import LabelEncoder

# --- CẤU HÌNH ---
input_dir = r"C:\Users\dotie\OneDrive\Documents\FAERS\Processed_Chunks"
all_files = glob.glob(os.path.join(input_dir, "processed_final_*.csv"))

# --- 1. XÂY DỰNG TỪ ĐIỂN ID (MAPPING) ---
# Để tiết kiệm RAM, ta chỉ quét tên cột (vốn chứa tên thuốc/bệnh) chứ không đọc toàn bộ dữ liệu
print("--- Bước 1: Quét tên thuốc và phản ứng để tạo ID ---")

# Đọc file đầu tiên để lấy danh sách cột (Vì các file đã đồng bộ cột)
df_cols = pd.read_csv(all_files[0], nrows=0) 
drug_names = [c for c in df_cols.columns if c.startswith('x ') and c not in ['x Age', 'x Gender', 'x AgeGroup']]
reaction_names = [c for c in df_cols.columns if c.startswith('y ')]

print(f"Số lượng thuốc: {len(drug_names)}")
print(f"Số lượng phản ứng: {len(reaction_names)}")

# --- 2. XÂY DỰNG DANH SÁCH CẠNH (EDGE LIST) ---
print("--- Bước 2: Đọc dữ liệu và tạo danh sách cạnh ---")

# Khởi tạo list chứa cạnh
src_patient_drug = []
dst_patient_drug = []
src_patient_react = []
dst_patient_react = []

patient_features_list = []
global_patient_id = 0 # ID bệnh nhân tăng dần

for f in all_files:
    # Đọc từng chunk
    print(f"Processing: {os.path.basename(f)}")
    df = pd.read_csv(f)
    
    num_rows = len(df)
    local_patient_ids = np.arange(num_rows) + global_patient_id
    
    # 1. Lấy đặc trưng bệnh nhân (Age, Gender)
    # Normalize Age: chia 100
    p_feats = df[['x Age', 'x Gender']].values
    p_feats[:, 0] = p_feats[:, 0] / 100.0 
    patient_features_list.append(p_feats)
    
    # 2. Tạo cạnh Patient -> Drug
    # Lấy ma trận con của thuốc (dạng dense)
    # Lưu ý: Nếu máy yếu